In [26]:
from pyspark.sql import SparkSession
# \
    
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("OrderKafkaProcessor") \
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0,org.apache.kafka:kafka-clients:4.0.0") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

spark.sparkContext .setLogLevel("WARN")

sc = spark.sparkContext

rdd = sc.parallelize(range(100 + 1))
rdd.sum()

5050

In [24]:
subscribeTopic = "ORDERS"

dfOrders = (
    spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers","broker01:29092,broker02:29092,broker03:29092")
    .option("subscribe", subscribeTopic)
    .option("startingOffsets","earliest")    
    .load()
)

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

In [ ]:

schema = StructType([
    StructField("ordertime", StringType()),
    StructField("orderid",  StringType()),
    StructField("itemid",  StringType()),
    StructField("city",  StringType()),
])

dfOrdersTest = (
    dfOrders
        .withColumn("json_data", from_json( col("value").cast("string"), schema))
        .withColumn("ordertime",  col("json_data.ordertime").cast("string"))
        .withColumn("orderid",  col("json_data.orderid").cast("string"))
        .withColumn("itemid",  col("json_data.itemid").cast("string"))
        .withColumn("address.city",  col("json_data.city").cast("string"))
        .drop(col("address"))
)

dfOrdersTest.show()



